Potentially Hazardous Asteroids (PHAs) are currently defined based on parameters that measure the asteroid's potential to make threatening close approaches to the Earth. Specifically, all asteroids with an Earth Minimum Orbit Intersection Distance (MOID) of 0.05 au or less and an absolute magnitude (H) of 22.0 or less are considered PHAs. In other words, asteroids that can't get any closer to the Earth (i.e., MOID) than 0.05 au (roughly 7,480,000 km or 4,650,000 mi) or are smaller than about 140 m (~500 ft) in diameter (i.e., H = 22.0 with assumed albedo of 14%) are not considered PHAs.

In [1]:
import pandas as pd
from datetime import timedelta, date
import configparser
import pickle

#libraries for api get requests
import requests
from urllib.request import urlretrieve
import json


In [2]:
# Setting Column widthsize to view all columns in pandas data frame
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
config = configparser.ConfigParser()
config.read('dl.cfg')
apiKey = config['NASA']['apikey']


# **NEO FEED**

### **Sample Output**
`{'absolute_magnitude_h': 19.88,
 'close_approach_data': [{'close_approach_date': '2021-11-10',
                          'close_approach_date_full': '2021-Nov-10 20:29',
                          'epoch_date_close_approach': 1636576140000,
                          'miss_distance': {'astronomical': '0.2306734775',
                                            'kilometers': '34508260.899492925',
                                            'lunar': '89.7319827475',
                                            'miles': '21442439.023835365'},
                          'orbiting_body': 'Earth',
                          'relative_velocity': {'kilometers_per_hour': '46525.6340756068',
                                                'kilometers_per_second': '12.9237872432',
                                                'miles_per_hour': '28909.219684368'}}],
 'estimated_diameter': {'feet': {'estimated_diameter_max': 2060.7486784504,
                                 'estimated_diameter_min': 921.5948259116},
                        'kilometers': {'estimated_diameter_max': 0.6281161771,
                                       'estimated_diameter_min': 0.2809020939},
                        'meters': {'estimated_diameter_max': 628.116177092,
                                   'estimated_diameter_min': 280.902093949},
                        'miles': {'estimated_diameter_max': 0.3902931771,
                                  'estimated_diameter_min': 0.174544415}},
 'id': '2068347',
 'is_potentially_hazardous_asteroid': True,
 'is_sentry_object': False,
 'links': {'self': 'http://www.neowsapp.com/rest/v1/neo/2068347?api_key=tydbiYxRDkZwIRlMQaAilILralTGaivulTvpTnht'},
 'name': '68347 (2001 KB67)',
 'nasa_jpl_url': 'http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2068347',
 'neo_reference_id': '2068347'}`

In [4]:
with open("rawdata.pickle", "rb") as f:   # Loading the data collected 
   raw = pickle.load(f)

In [25]:
df = pd.DataFrame(raw)

In [29]:
df.isnull().sum()

links                                0
id                                   0
neo_reference_id                     0
name                                 0
nasa_jpl_url                         0
absolute_magnitude_h                 0
estimated_diameter                   0
is_potentially_hazardous_asteroid    0
close_approach_data                  0
is_sentry_object                     0
dtype: int64

In [27]:
#Droping Null Values
df.dropna(inplace=True)

In [31]:
# Getting Data from nested dictionaries of certain columns

df['estimated_diameter_min_km'] = df['estimated_diameter'].apply(lambda x: x['kilometers']['estimated_diameter_min'])

df['estimated_diameter_max_km'] = df['estimated_diameter'].apply(lambda x: x['kilometers']['estimated_diameter_max'])

df['close_approach_date'] = df['close_approach_data'].apply(lambda x: x[0]['close_approach_date'])

df['miss_distance_au'] = df['close_approach_data'].apply(lambda x: x[0]['miss_distance']['astronomical'])

df['relative_velocity_kmps'] = df['close_approach_data'].apply(lambda x: x[0]['relative_velocity']['kilometers_per_second'])

df['orbiting_body'] = df['close_approach_data'].apply(lambda x: x[0]['orbiting_body'])

In [32]:
df.dtypes

links                                 object
id                                    object
neo_reference_id                      object
name                                  object
nasa_jpl_url                          object
absolute_magnitude_h                 float64
estimated_diameter                    object
is_potentially_hazardous_asteroid       bool
close_approach_data                   object
is_sentry_object                        bool
estimated_diameter_min_km            float64
estimated_diameter_max_km            float64
close_approach_date                   object
miss_distance_au                      object
relative_velocity_kmps                object
orbiting_body                         object
dtype: object

In [33]:
df.head(4)

,links,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,estimated_diameter,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,estimated_diameter_min_km,estimated_diameter_max_km,close_approach_date,miss_distance_au,relative_velocity_kmps,orbiting_body
0,{'self': 'http://www.neowsapp.com/rest/v1/neo/...,2053789,2053789,53789 (2000 ED104),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2053789,17.33,{'kilometers': {'estimated_diameter_min': 0.90...,True,"[{'close_approach_date': '2002-10-09', 'close_...",False,0.908981,2.032544,2002-10-09,0.1657650913,21.5748905048,Earth
1,{'self': 'http://www.neowsapp.com/rest/v1/neo/...,2338292,2338292,338292 (2002 UA31),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2338292,19.02,{'kilometers': {'estimated_diameter_min': 0.41...,False,"[{'close_approach_date': '2002-10-09', 'close_...",False,0.417402,0.933340,2002-10-09,0.2255639049,17.3257341038,Earth
2,{'self': 'http://www.neowsapp.com/rest/v1/neo/...,3251590,3251590,(2004 RC80),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3251590,24.40,{'kilometers': {'estimated_diameter_min': 0.03...,False,"[{'close_approach_date': '2002-10-09', 'close_...",False,0.035039,0.078350,2002-10-09,0.4332412059,17.0376265666,Earth
3,{'self': 'http://www.neowsapp.com/rest/v1/neo/...,3255465,3255465,(2004 SC56),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3255465,22.70,{'kilometers': {'estimated_diameter_min': 0.07...,False,"[{'close_approach_date': '2002-10-09', 'close_...",False,0.076658,0.171412,2002-10-09,0.0408540265,11.1709911789,Earth


In [34]:
#df_na4[['year','month','day']] = df_na4[['year','month','day']].apply(pd.to_numeric)
df[['id','neo_reference_id','miss_distance_au','relative_velocity_kmps',]] =df[['id','neo_reference_id','miss_distance_au','relative_velocity_kmps',]].apply(pd.to_numeric)

In [35]:
df[['close_approach_date']] = df[['close_approach_date']].apply(pd.to_datetime)
#df['miss_distance_au'] = df['miss_distance_au'].apply(lambda x: round(x,2))
#df['relative_velocity_kmps'] = df['relative_velocity_kmps'].apply((lambda x: round(x,2)))
#df['absolute_magnitude_h'] = df['absolute_magnitude_h'].apply(lambda x: round(x,2))

In [36]:
df.dtypes

links                                        object
id                                            int64
neo_reference_id                              int64
name                                         object
nasa_jpl_url                                 object
absolute_magnitude_h                        float64
estimated_diameter                           object
is_potentially_hazardous_asteroid              bool
close_approach_data                          object
is_sentry_object                               bool
estimated_diameter_min_km                   float64
estimated_diameter_max_km                   float64
close_approach_date                  datetime64[ns]
miss_distance_au                            float64
relative_velocity_kmps                      float64
orbiting_body                                object
dtype: object

In [37]:
clean = df.drop(columns=(['links','neo_reference_id','nasa_jpl_url','estimated_diameter','close_approach_data']))
clean.tail(5)

,id,name,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_min_km,estimated_diameter_max_km,close_approach_date,miss_distance_au,relative_velocity_kmps,orbiting_body
80587,54225724,(2021 XD),25.210,False,False,0.024130,0.053956,2021-11-28,0.071978,6.548339,Earth
80588,54226110,(2021 XM),27.122,False,False,0.010004,0.022369,2021-11-28,0.003351,8.748354,Earth
80589,54226410,(2021 XK1),26.396,False,False,0.013975,0.031249,2021-11-28,0.025174,8.852189,Earth
80590,54226786,(2021 XX1),27.741,False,False,0.007522,0.016821,2021-11-28,0.018287,8.581292,Earth
80591,54226789,(2021 XZ1),23.563,False,False,0.051518,0.115197,2021-11-28,0.248633,14.557350,Earth


In [38]:
clean.dtypes

id                                            int64
name                                         object
absolute_magnitude_h                        float64
is_potentially_hazardous_asteroid              bool
is_sentry_object                               bool
estimated_diameter_min_km                   float64
estimated_diameter_max_km                   float64
close_approach_date                  datetime64[ns]
miss_distance_au                            float64
relative_velocity_kmps                      float64
orbiting_body                                object
dtype: object

# **Code Above**
---
